In [1]:
import requests

In [65]:
url = 'https://www.bok.or.kr/portal/singl/newsData/listCont.do?'
params = {
    'pageIndex': 1,
    'targetDepth': 4,
    'menuNo': 200789,
    'syncMenuChekKey': 1,
    'depthSubMain': '',
    'subMainAt': '',
    'searchCnd': 1,
    'searchKwd': '',
    'depth2': 200038,
    'depth3': 201154,
    'depth4': 200789,
}
res = requests.get(url, params=params)
res.status_code

200

In [46]:
from bs4 import BeautifulSoup as bs

In [66]:
soup = bs(res.text)
soup.prettify

<bound method Tag.prettify of <html><head><script>
//@@@1@@@
  $(function() {
    if (false){
      $('.date-set').show();
    }else{
      $('.date-set').hide();
    }

    $(".select1 select").on("change", function(){
      var result = $(".select1 option:selected").val()
      if(result == 5){
        $('.date-set').show();
      }else{
        $('.date-set').hide();
      }
    });
  });

  // 콘텐츠 서비스
  $(function () {
    // 달력 언어설정
    if (typeof $.fn.datepicker != 'undefined') {
      $.fn.datepicker.dates.ko = {
        days: ['일요일', '월요일', '화요일', '수요일', '목요일', '금요일', '토요일'],
        daysShort: ['일', '월', '화', '수', '목', '금', '토'],
        daysMin: ['일', '월', '화', '수', '목', '금', '토'],
        months: ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'],
        monthsShort: ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'],
        today: '오늘',
        clear: '삭제',
        format: 'yyyy-mm-dd',
        titleFormat: 'yyyy년mm월',
   

In [70]:
lis = soup.select('li.bbsRowCls')
links = [li.select_one('a').get('href') for li in lis]
print(links)

['/portal/bbs/B0000245/view.do?nttId=10086296&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=200789', '/portal/bbs/B0000245/view.do?nttId=10086177&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=200789', '/portal/bbs/B0000245/view.do?nttId=10085855&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=200789', '/portal/bbs/B0000245/view.do?nttId=10085535&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=200789', '/portal/bbs/B0000245/view.do?nttId=10084604&searchCnd=1&searchKwd=&depth2=200038&depth3=201154&depth4=200789&depth=200789&pageUnit=10&pageIndex=1&programType=newsData&menuNo=200789&oldMenuNo=200

In [71]:
pdf_url = 'https://www.bok.or.kr' + links[0]
res = requests.get(pdf_url)
res.status_code

200

In [73]:
soup = bs(res.text)

In [76]:
pdfs = soup.select('a.file')
a = 'https://www.bok.or.kr' + pdfs[1].get('href')
a

'https://www.bok.or.kr/fileSrc/portal/8426eabe5ead44c4a1cfe6eccbc8ed3d/2/0cdeedba97704d7d93f46249561cc473.pdf'

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from scrapy_selenium import SeleniumRequest
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from PyPDF2 import PdfReader
from io import BytesIO
import os
import time

ModuleNotFoundError: No module named 'scrapy_selenium'

In [98]:
call_url = 'https://www.korcham.net/nCham/Service/EconBrief/appl/ProspectBoardList.asp'
params = {
    'board_type': 1,
    'pageno': 1,
    'daybt': 'OldNow',
    'm_OldDate': 20140811,
    'm_NowDate': 20240811
}
call_res = requests.get(call_url, params=params)
call_res.encoding = 'euc-kr'
call_res.status_code

200

In [99]:
call_soup = bs(call_res.text)
call_soup

<!DOCTYPE html>
<html class="ko" lang="ko">
<head>
<!-- head area -->
<meta charset="utf-8"/> <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/> <title>코참넷</title> <script type="application/ld+json">           
{                                               
 "@context": "http://schema.org",           
 "@type": "Organization",                         
 "name": "대한상공회의소",                            
 "url": "https://www.korcham.net",          
 "sameAs": [                                  
   "https://www.youtube.com/channel/UCtsmQ4ln18jB-rtyGQD6yXw"    
 ]                                              
}                                               
</script> <!-- 페이지 로딩 form element styling plugin 으로 인해 화면 가림 end --> <!--<link rel="shortcut icon" href="favicon.ico">--> <script src="//ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js"></script> <script>window.jQuery || document.write(unescape('%3Cscript src="//ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.mi

In [100]:
table_tag = call_soup.select_one('div.tablewrap')

In [101]:
column_names = []
columns = table_tag.select('thead th')
for column in columns:
    column_names.append(column.text)
column_names

['날짜', '콜금리', 'CD(91일)', '국고채(3년)', '국고채(5년)', '회사채(3년,AA-)']

In [102]:

rows = []
datas = table_tag.select('tbody tr')
for data in datas:
    row = [da.text for da in data.select('td')]
    rows.append(row)
rows

[['2024.08.09', '3.53', '3.47', '2.94', '-', '3.41'],
 ['2024.08.08', '3.47', '3.48', '2.91', '-', '3.38'],
 ['2024.08.07', '3.53', '3.48', '2.93', '-', '3.40'],
 ['2024.08.06', '3.54', '3.47', '2.94', '-', '3.40'],
 ['2024.08.05', '3.54', '3.48', '2.81', '-', '3.27'],
 ['2024.08.02', '3.51', '3.48', '2.94', '-', '3.39'],
 ['2024.08.01', '3.53', '3.49', '2.98', '-', '3.43'],
 ['2024.07.31', '3.51', '3.50', '3.00', '-', '3.46'],
 ['2024.07.30', '3.5', '3.50', '2.99', '-', '3.45'],
 ['2024.07.29', '3.54', '3.50', '2.98', '-', '3.43'],
 ['2024.07.26', '3.55', '3.50', '3.00', '-', '3.48'],
 ['2024.07.25', '3.52', '3.51', '3.02', '-', '3.48'],
 ['2024.07.24', '3.55', '3.51', '3.05', '-', '3.51'],
 ['2024.07.23', '3.51', '3.52', '3.08', '-', '3.53'],
 ['2024.07.22', '3.56', '3.52', '3.07', '-', '3.52']]

In [110]:
interest_url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do'
interest_params = {
    'dataSeCd': '01',
    'menuNo': '200643'
}
interest_res = requests.get(interest_url, params=interest_params)
interest_res.status_code

200

In [111]:
int_soup = bs(interest_res.text)
int_soup

<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title> | 한국은행 기준금리 추이(목록) | 통화정책방향 | 통화정책 | 정책/업무 |  한국은행 홈페이지</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="한국은행 기준금리 추이" property="title"/>
<meta content="한국은행" property="main"/>
<meta content="" property="creator"/>
<meta content="한국은행" property="publisher"/>
<meta content=" | 한국은행 기준금리 추이(목록) | 통화정책방향 | 통화정책 | 정책/업무 |  한국은행 홈페이지" property="subject"/>
<meta content="" property="keyword"/>
<meta content="" property="description"/>
<link href="https://static-cdn.bok.or.kr/static/commons/bok.ico" rel="icon" type="image/x-icon"/>
<link href="https://static-cdn.bok.or.kr/static/portal/css/sub.css" rel="stylesheet"/>
<script src="https://static-cdn.bok.or.kr/static/jslibrary/jquery-3.5.1.min.js"></script>
<script src="https://static-cdn.bok.or.kr/static/jslibrary/bootstrap/bootstrap.min.js"></script>
<script src="https://sta

In [116]:
int_soup.select_one('div.table tbody tr').text.split()

['2023', '01월', '13일', '3.50']